In [59]:
# Imports
import pandas as pd
import numpy as np
import nltk 
import matplotlib.pyplot as plt
import csv
import random

In [63]:
reclustered_monthly_data = pd.read_csv("output_hpc/reclustered_monthly_data_final.csv", sep = "\t", encoding = "utf-8", quoting = csv.QUOTE_NONE )

In [64]:
data = reclustered_monthly_data.head(100)

In [74]:
n_rows = data.shape[0]

In [75]:
data["A"] = np.random.rand(n_rows)
data["F"] = np.random.rand(n_rows)
data["N"] = np.random.rand(n_rows)

/tmp/ipykernel_9866/686247784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["A"] = np.random.rand(n_rows)
/tmp/ipykernel_9866/686247784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["F"] = np.random.rand(n_rows)
/tmp/ipykernel_9866/686247784.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [76]:
data_avg = data.groupby("user")[['A', 'F', 'N']].mean().reset_index()

In [77]:
max_val = data_avg.iloc[1][["A", "F", "N"]].max()
max_col = data_avg.iloc[1][["A", "F"]].idxmax()
print(max_val *-1)
print(max_val, max_col)

-0.974251290632651
0.974251290632651 F


In [78]:

def get_stance(row): 
    if row["N"] == 1.0: 
        return 0
    elif row["A"] > row["F"]:
        return row["A"]
    elif row["F"] > row["A"]:
        return row["F"]*-1
    else: 
        return random.choice([row["A"], row["F"]])

        # max_val = row[["A", "F"]].idxmax()
        # if max_val == "A":
        #     return row["A"]
        # elif max_val == "F": 
        #     return row["F"]*-1
        # else: 
        #     return "ERROR"


In [79]:
data_avg["stance"] = data_avg.apply(lambda x: get_stance(x), axis = 1)

In [80]:
data_avg

,user,A,F,N,stance
0,AlbionPrince,0.421794,0.818319,0.983197,-0.818319
1,BenFoldsFourLoko,0.424729,0.974251,0.620601,-0.974251
2,Betrashndie,0.863502,0.903815,0.055659,-0.903815
3,BloodWiz,0.363536,0.455848,0.748766,-0.455848
4,Call_Me_Hurr1cane,0.781419,0.370661,0.469383,0.781419
...,...,...,...,...,...
91,thebigslapper,0.130504,0.618875,0.491716,-0.618875
92,throwrowrowawayyy,0.761511,0.540064,0.221474,0.761511
93,trogon,0.552259,0.017077,0.725244,0.552259
94,weeteacups,0.327439,0.360535,0.405771,-0.360535
